In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import csv

with open("MSHSAA Swimming Performance List.html", "r", encoding="utf-8") as f:
    soup = BeautifulSoup(f, "html.parser")

# analyze HTML structure

In [2]:
# Find tables
tables = soup.find_all('table')
print(f"Number of tables: {len(tables)}")

Number of tables: 13


In [3]:
# Look for specific class patterns
fs_grid_elements = soup.find_all(class_='fs_grid')
print(f"Elements with class 'fs_grid': {len(fs_grid_elements)}")

Elements with class 'fs_grid': 11


In [4]:
# Check what's inside fs_grid elements
for i, element in enumerate(fs_grid_elements[:3]):
    print(f"--- fs_grid element {i+1} ---")
    print(f"Tag: {element.name}")
    print(f"Contains {len(element.find_all('tr'))} rows")
    
    # Look at first few rows
    rows = element.find_all('tr')
    for j, row in enumerate(rows[:3]):
        print(f"\nRow {j}:")
        tds = row.find_all('td')
        for k, td in enumerate(tds):
            print(f"  td[{k}]: {td.text.strip()[:50]}...")

--- fs_grid element 1 ---
Tag: table
Contains 90 rows

Row 0:
  td[0]: Rank...
  td[1]: School...
  td[2]: Contest...
  td[3]: Date...
  td[4]: Time...

Row 1:
  td[0]: #1...
  td[1]: St. Joseph's Academy...
  td[2]: COMO Invite...
  td[3]: 1/9/26...
  td[4]: 1:46.57...

Row 2:
  td[0]: #2...
  td[1]: Park Hill South...
  td[2]: COMO Invitational...
  td[3]: 1/9/26...
  td[4]: 1:48.32...
--- fs_grid element 2 ---
Tag: table
Contains 180 rows

Row 0:
  td[0]: Rank...
  td[1]: Student...
  td[2]: Grade...
  td[3]: School...
  td[4]: Contest...
  td[5]: Date...
  td[6]: Time...

Row 1:
  td[0]: #1...
  td[1]: Helena Tietjen
Pembroke Hill...
  td[2]: 12...
  td[3]: Pembroke Hill...
  td[4]: COMO Invitational...
  td[5]: 1/9/26...
  td[6]: 1:48.61...

Row 2:
  td[0]: #2...
  td[1]: Anna-Grace Guenther
St. Joseph's Academy...
  td[2]: 12...
  td[3]: St. Joseph's Academy...
  td[4]: COMO Invite...
  td[5]: 1/9/26...
  td[6]: 1:51.91...
--- fs_grid element 3 ---
Tag: table
Contains 153 rows

R

In [6]:
import pandas as pd
from bs4 import BeautifulSoup
import csv

# List of events in the exact order they appear on the page
EVENT_ORDER = [
    "200 Medley Relay",
    "200 Free",
    "200 IM",
    "50 Free",
    "100 Fly",
    "100 Free",
    "500 Free",
    "200 Free Relay",
    "100 Back",
    "100 Breast",
    "400 Free Relay"
]

def clean_time(time_str):
    """Remove * (record marker) and normalize whitespace so times don't break CSV."""
    if not time_str:
        return time_str
    # cleaned = time_str.replace('*', '').replace('\n', ' ').replace('\r', ' ')
    cleaned = time_str.replace('*', '')
    return ' '.join(cleaned.split()).strip()

def scrape_all_events_with_known_order(html_file_path, output_csv_path):
    """
    Scrape all swimming events from the HTML file
    Uses the known event order from EVENT_ORDER list
    """
    with open(html_file_path, 'r', encoding='utf-8') as file:
        html_content = file.read()
    
    soup = BeautifulSoup(html_content, 'html.parser')
    all_data = []
    
    # Find all event tables (fs_grid elements)
    fs_grid_tables = soup.find_all('table', class_='fs_grid')
    
    print(f"Found {len(fs_grid_tables)} event tables")
    print(f"Expected {len(EVENT_ORDER)} events based on your list")
    
    # Make sure there's the right number of tables
    if len(fs_grid_tables) != len(EVENT_ORDER):
        print(f"Warning: Found {len(fs_grid_tables)} tables but expected {len(EVENT_ORDER)} events")
        print("Using available events in order...")
    
    # Process each fs_grid table and assign event names from the list
    for table_index, table in enumerate(fs_grid_tables):
        # Get event name from the predefined list
        if table_index < len(EVENT_ORDER):
            event_name = EVENT_ORDER[table_index]
        else:
            event_name = f"Event {table_index + 1}"
        
        print(f"\nProcessing {event_name} (table {table_index + 1}/{len(fs_grid_tables)})")
        
        # Get all rows in this table
        rows = table.find_all('tr')
        print(f"  Table has {len(rows)} rows (including header)")
        
        # Determine if this event is a relay based on event name
        is_relay_event = "Relay" in event_name
        
        # Skip the header row
        for row_index, row in enumerate(rows):
            if row_index == 0:  # Skip header row
                continue
            
            tds = row.find_all('td')
            if not tds:  # Skip empty rows
                continue
            
            # Determine if this is a relay table based on structure
            is_relay_table = len(tds) == 5  # Relay tables have 5 columns
            
            # Validate: Event name should match table structure
            if is_relay_event != is_relay_table:
                print(f"  Warning: Event '{event_name}' structure mismatch!")
                print(f"    Event suggests relay: {is_relay_event}")
                print(f"    Table structure suggests relay: {is_relay_table}")
            
            if is_relay_table:
                # Relay structure: Rank, School, Contest, Date, Time
                seed_rank = tds[0].text.strip().replace('#', '')
                team = tds[1].text.strip()
                
                # For relays, name is the same as team
                name = team
                
                # Extract contest/meet
                contest = tds[2].text.strip() if len(tds) > 2 else ""
                
                # Extract date
                date = tds[3].text.strip() if len(tds) > 3 else ""
                
                # Extract time
                best_time = tds[4].text.strip() if len(tds) > 4 else ""
                
                is_relay = True
                grade = ""  # No grade for relays
                
            else:
                # Individual event structure: Rank, Student, Grade, School, Contest, Date, Time
                seed_rank = tds[0].text.strip().replace('#', '')
                
                # Name and team (mobile version in td[1])
                name_cell = tds[1]
                name_text = name_cell.get_text(separator='\n', strip=True)
                
                # Extract name (first line)
                lines = [line.strip() for line in name_text.split('\n') if line.strip()]
                if lines:
                    name = lines[0]
                else:
                    name = name_cell.text.strip()
                
                # Grade
                grade = tds[2].text.strip() if len(tds) > 2 else ""
                
                # Team (desktop version from td[3])
                team = tds[3].text.strip() if len(tds) > 3 else ""
                
                # If team is empty, try to extract from name cell (mobile version)
                if not team and len(lines) > 1:
                    team = lines[1]
                
                # Extract contest/meet
                contest = tds[4].text.strip() if len(tds) > 4 else ""
                
                # Extract date
                date = tds[5].text.strip() if len(tds) > 5 else ""
                
                # Extract time
                best_time = tds[6].text.strip() if len(tds) > 6 else ""
                
                is_relay = False
            
            # Clean up name (remove extra whitespace)
            name = ' '.join(name.split())
            team = ' '.join(team.split())
            best_time = clean_time(best_time)
            
            # Create data record
            record = {
                'name': name,
                'team': team,
                'event': event_name,
                'best_time': best_time,
                'seed_rank': seed_rank,
                'is_relay': is_relay,
                'grade': grade,
                'contest': contest,
                'date': date,
                'table_index': table_index + 1
            }
            
            all_data.append(record)
    
    # Convert to DataFrame
    df = pd.DataFrame(all_data)
    
    # Reorder columns to match your requested format
    df = df[['name', 'team', 'event', 'best_time', 'seed_rank', 'is_relay']]
    
    # Write to CSV
    df.to_csv(output_csv_path, index=False)
    print(f"\nScraped {len(all_data)} total records")
    print(f"Saved to {output_csv_path}")
    
    return df

def verify_event_counts(df):
    """
    Verify each event has the right number of records
    """
    print("\n=== Event Count Verification ===")
    
    event_counts = df.groupby('event').size().reset_index(name='count')
    event_counts['is_relay'] = df.groupby('event')['is_relay'].first().values
    
    # Add expected order
    event_order_dict = {event: i for i, event in enumerate(EVENT_ORDER)}
    event_counts['order'] = event_counts['event'].map(event_order_dict)
    event_counts = event_counts.sort_values('order')
    
    print("Records per event:")
    for _, row in event_counts.iterrows():
        relay_flag = "(Relay)" if row['is_relay'] else "(Individual)"
        print(f"  {row['event']:20} {relay_flag:12} {row['count']:4} records")
    
    total_records = event_counts['count'].sum()
    print(f"\nTotal records across all events: {total_records}")
    
    return event_counts

def display_sample_data(df, num_samples=5):
    """
    Display sample data from each event
    """
    print("\n=== Sample Data from Each Event ===")
    
    for event in EVENT_ORDER:
        if event in df['event'].values:
            event_data = df[df['event'] == event]
            print(f"\n{event}:")
            print("-" * 80)
            
            samples = event_data.head(num_samples)
            for _, row in samples.iterrows():
                relay_flag = "[Relay]" if row['is_relay'] else "[Individual]"
                print(f"  Rank {row['seed_rank']:>3} | {row['name'][:30]:30} | "
                      f"{row['team'][:20]:20} | {row['best_time']:>8} {relay_flag}")

def main():
    html_file = "MSHSAA Swimming Performance List.html"
    output_file = "swimming_performance.csv"
    
    print("=" * 80)
    print("MSHSAA Swimming Performance Scraper")
    print("=" * 80)
    print(f"Processing: {html_file}")
    print(f"Output: {output_file}")
    print(f"\nEvent List ({len(EVENT_ORDER)} events):")
    for i, event in enumerate(EVENT_ORDER, 1):
        relay_flag = "(Relay)" if "Relay" in event else "(Individual)"
        print(f"  {i:2}. {event:20} {relay_flag}")
    
    print("\n" + "=" * 80)
    print("Starting scrape...")
    
    # Run the scraper
    df = scrape_all_events_with_known_order(html_file, output_file)
    
    # Verify results
    event_counts = verify_event_counts(df)
    
    # Display samples
    display_sample_data(df)
    
    # Summary statistics
    print("\n" + "=" * 80)
    print("SUMMARY STATISTICS")
    print("=" * 80)
    print(f"Total records: {len(df)}")
    print(f"Relay records: {df['is_relay'].sum()}")
    print(f"Individual records: {len(df) - df['is_relay'].sum()}")
    print(f"\nUnique swimmers: {df[~df['is_relay']]['name'].nunique()}")
    print(f"Unique teams: {df['team'].nunique()}")

if __name__ == "__main__":
    main()

MSHSAA Swimming Performance Scraper
Processing: MSHSAA Swimming Performance List.html
Output: swimming_performance.csv

Event List (11 events):
   1. 200 Medley Relay     (Relay)
   2. 200 Free             (Individual)
   3. 200 IM               (Individual)
   4. 50 Free              (Individual)
   5. 100 Fly              (Individual)
   6. 100 Free             (Individual)
   7. 500 Free             (Individual)
   8. 200 Free Relay       (Relay)
   9. 100 Back             (Individual)
  10. 100 Breast           (Individual)
  11. 400 Free Relay       (Relay)

Starting scrape...
Found 11 event tables
Expected 11 events based on your list

Processing 200 Medley Relay (table 1/11)
  Table has 90 rows (including header)

Processing 200 Free (table 2/11)
  Table has 180 rows (including header)

Processing 200 IM (table 3/11)
  Table has 153 rows (including header)

Processing 50 Free (table 4/11)
  Table has 170 rows (including header)

Processing 100 Fly (table 5/11)
  Table has 145 ro

# Import dive performance data

Parse MSHSAA Dive Performance Listing.html: two tables (Class 1, Class 2). Output `dive_performance.csv` and optionally `class1_dive.csv` / `class2_dive.csv`.

In [7]:
import re

def scrape_dive_performance(html_path="MSHSAA Dive Performance Listing.html", output_path="dive_performance.csv"):
    """Parse dive tables from MSHSAA HTML. First table = Class 1, second = Class 2."""
    with open(html_path, "r", encoding="utf-8") as f:
        soup = BeautifulSoup(f, "html.parser")

    tables = soup.find_all("table", class_="fs_grid")
    if len(tables) < 2:
        print(f"Warning: expected 2 dive tables (Class 1 & 2), found {len(tables)}")

    rows_all = []
    for table_idx, table in enumerate(tables):
        class_num = table_idx + 1  # 1 or 2
        for tr in table.find_all("tr"):
            if "hide" in (tr.get("class") or []):
                continue
            tds = tr.find_all("td")
            if len(tds) < 5:
                continue
            rank_text = tds[0].get_text(strip=True)
            if not rank_text or not rank_text.replace("#", "").isdigit():
                continue  # skip header
            # Student: name on first line, school in div.small.gray (or second line)
            student_cell = tds[1]
            lines = [l.strip() for l in student_cell.get_text(separator="\n", strip=True).split("\n") if l.strip()]
            name = lines[0] if lines else ""
            team = lines[1] if len(lines) > 1 else ""
            if not team:
                school_div = student_cell.find("div", class_=lambda c: c and ("gray" in (c if isinstance(c, list) else [])))
                if school_div:
                    team = school_div.get_text(strip=True)
            name = " ".join(name.split())
            team = " ".join(team.split())
            # Score: first number in cell (e.g. 418.40)
            score_text = tds[3].get_text(strip=True)
            m = re.search(r"[\d.]+", score_text)
            total_score = float(m.group()) if m else None
            if total_score is None:
                continue
            try:
                difficulty = float(tds[4].get_text(strip=True))
            except (ValueError, IndexError):
                difficulty = None
            rows_all.append({
                "name": name,
                "team": team,
                "total_score": total_score,
                "difficulty": difficulty,
                "class": class_num,
            })

    df = pd.DataFrame(rows_all)
    df = df.sort_values(["class", "total_score"], ascending=[True, False]).reset_index(drop=True)
    df.to_csv(output_path, index=False)
    print(f"Saved {len(df)} dive records to {output_path}")
    print(f"  Class 1: {len(df[df['class']==1])}, Class 2: {len(df[df['class']==2])}")

    # Optional: class-specific CSVs
    for c in [1, 2]:
        subset = df[df["class"] == c]
        if not subset.empty:
            subset.to_csv(f"class{c}_dive.csv", index=False)
            print(f"  class{c}_dive.csv: {len(subset)} rows")
    return df

df_dive = scrape_dive_performance()
df_dive.head(10)

Saved 94 dive records to dive_performance.csv
  Class 1: 41, Class 2: 53
  class1_dive.csv: 41 rows
  class2_dive.csv: 53 rows


,name,team,total_score,difficulty,class
0,Naya Narciso,Notre Dame de Sion,418.4011,22.6,1
1,Journey Wildschuetz,Central (Springfield),404.2511,22.3,1
2,Stella Henderson,Notre Dame de Sion,379.7511,21.0,1
3,Clara Hugge,Visitation Academy,369.2011,20.1,1
4,Ella Ball,Father Tolton,361.8011,21.2,1
5,Hayden Duffy,Visitation Academy,349.4511,20.7,1
6,Gabriella Pupillo,Parkway West,334.1511,20.2,1
7,Evelyn Behrens,Lincoln College Prep,332.3511,21.0,1
8,Eloise Sweeny,St. Teresa's Academy,330.8511,20.2,1
9,Grace Gunter,St. Teresa's Academy,326.6011,20.8,1
